# Customer LifeTime Value (CLV)

Customer lifetime value is the total worth to a business of a customer over the whole period of their relationship. It’s an important metric as it costs less to keep existing customers than it does to acquire new ones, so increasing the value of your existing customers is a great way to drive growth. Knowing the CLV helps businesses develop strategies to acquire new customers and retain existing ones while maintaining profit margins. <br><br>
CLV=Expected No. of Transaction * Revenue per Transaction * Margin <br>
Where;<br>
Expected No. of Transaction is calculated using BG/NBD Model<br>
Revenue per Transaction is calculated using Gama Gama Model and<br> 
Margin is provided by historical transaction or we can take a standard value of 5%.<br><br>
Model<br>
For this use-case we will use the lifetimes library. 
Lifetimes is used to analyze your users based on a few assumption:<br>

1. Users interact with you when they are "alive".<br>
2. Users under study may "die" after some period of time.

<a href="https://lifetimes.readthedocs.io/en/master/index.html"> more on library</a>
<br><br>
Datasets<br>
We will use the online retail  transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers <a href="http://archive.ics.uci.edu/ml/datasets/online+retail">more on dataset</a>.



In [1]:
# !pip install Lifetimes==0.11.3

Import libraries

In [2]:
# Dash dependencies import
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.tools import mpl_to_plotly
px.defaults.template = "ggplot2"

from lifetimes.utils import *
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.plotting import plot_frequency_recency_matrix
from lifetimes.plotting import plot_probability_alive_matrix
from lifetimes.plotting import plot_period_transactions
from lifetimes.plotting import plot_history_alive

Load data

In [3]:
df=pd.read_csv("../datasets/Customer Lifetime Value Online Retail.csv",encoding="cp1252")

Explore Data

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


In [5]:
df.shape

(541909, 8)

Check missing data

In [6]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Drop records with missing CustomerID

In [7]:
df = df[pd.notnull(df['CustomerID'])]
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Remove negative quantities

In [8]:
df=df[df['Quantity']>0]

Convert CustomerID to String

In [9]:
df['CustomerID'] = df['CustomerID'].astype(int)
df['CustomerID'] = df['CustomerID'].astype(str) 

Add Total Sales Column

In [10]:
df['TotalSales']=df['Quantity']*df['UnitPrice']
df['TotalSales']=round(df['TotalSales'],2)

Extract Data, Month and Year from InvoiceDate

In [ ]:
df['Date'] = pd.to_datetime(df['InvoiceDate'], format="%d/%m/%Y %H:%M").dt.date
df['Month'] = pd.to_datetime(df['InvoiceDate'], format="%d/%m/%Y %H:%M").dt.strftime('%Y-%m')
df['Year'] = pd.to_datetime(df['InvoiceDate'], format="%d/%m/%Y %H:%M").dt.year
df['Day'] = pd.to_datetime(df['InvoiceDate'], format="%d/%m/%Y %H:%M").dt.day

In [ ]:
df.head()

Save processed data

In [ ]:
df.to_csv("../datasets/Customer Lifetime Value Online Retail Processed.csv")

Total Transactions

In [ ]:
df.shape[0]

Unique Customers

In [ ]:
df['CustomerID'].nunique()

Total Revenue

In [ ]:
df['TotalSales'].sum()

Get Statistical Summary

In [ ]:
df.describe()

In [ ]:
df.shape

Customers Distribution per Country

In [ ]:
customer_count_df=df.groupby( ["Country"], as_index=False )["CustomerID"].count().sort_values(by="CustomerID",ascending=False)
customer_count_df.columns=['Country','Customers']
customer_count_df.head(10)

In [ ]:
customer_count_df=customer_count_df[customer_count_df['Country']!='United Kingdom']
fig=px.bar(customer_count_df.head(10),x='Country',y='Customers',text='Customers',color='Country',title='Top 10 Customers Distribution per Country')
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.8),autosize=True,margin=dict(t=30,b=0,l=0,r=0))

Revenue Distribution per Country

In [ ]:
revenue_per_country_df=df.groupby( ["Country"], as_index=False )["TotalSales"].sum().sort_values(by="TotalSales",ascending=False)
revenue_per_country_df.columns=['Country','Revenue']
revenue_per_country_df.head(10)

In [ ]:
revenue_per_country_df=revenue_per_country_df[revenue_per_country_df['Country']!='United Kingdom']
fig=px.bar(revenue_per_country_df.head(10),x='Country',y='Revenue',text='Revenue',color='Country',title='Top 10 Countries by Revenue')
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.80),autosize=True,margin=dict(t=30,b=0,l=0,r=0))

Customer with hightest Transactions

In [ ]:
customer_transaction_df=df.groupby( ["CustomerID"], as_index=False )["Quantity"].count().sort_values(by="Quantity",ascending=False)
customer_transaction_df.columns=['Customers','Count']
customer_transaction_df=customer_transaction_df[customer_transaction_df['Customers']!='nan']
customer_transaction_df.head(10)

In [ ]:
# customer_transaction_df=customer_transaction_df[customer_transaction_df['Customers']!=np.nan]
# customer_transaction_df=customer_transaction_df.drop(np.nan)
fig=px.bar(customer_transaction_df.head(10),x='Customers',y='Count',color='Customers',text='Count',
title='Customer Count')
fig.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.80),autosize=True,margin=dict(t=30,b=0,l=0,r=0))

Top 10 Customers with highest Revenue spend

In [ ]:
revenue_per_customers_df=df.groupby('CustomerID', as_index=False )['TotalSales'].sum().sort_values(by="TotalSales",ascending=False)
revenue_per_customers_df.columns=['Customers','Revenue']
revenue_per_customers_df=revenue_per_customers_df[revenue_per_customers_df['Customers']!='nan']
revenue_per_customers_df.head(10)

In [ ]:
fig=px.bar(revenue_per_customers_df.head(10),x='Customers',y='Revenue',color='Customers',text='Revenue',
title='Customer Revenue')
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.80),autosize=True,margin=dict(t=30,b=0,l=0,r=0))

Distribution by Invoice Number

In [ ]:
df.InvoiceNo.value_counts().sort_values(ascending=False).head(10)

Top 10 Invoices with Highest Revenue Spend on

In [ ]:
revenue_per_invoice_df=df.groupby('InvoiceNo')['TotalSales'].sum().sort_values(ascending=False).head(10)
revenue_per_invoice_df

Stocks with most transactions

In [ ]:
df.StockCode.value_counts().sort_values(ascending=False).head(10)

Stocks with Highest Revenue

In [ ]:
revenue_per_stock_df=df.groupby('StockCode')['TotalSales'].sum().sort_values(ascending=False).head(10)
revenue_per_stock_df

Revenue Trend

In [ ]:
revenue_trend_df=df.groupby('Date', as_index=False )['TotalSales'].sum().sort_values(by="Date",ascending=True)
revenue_trend_df.columns=['Date','Revenue']
revenue_trend_df.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=revenue_trend_df['Date'], y=revenue_trend_df['Revenue'],name='Revenue',
                                line = dict(color='teal', width=2),line_shape='spline'))
fig.update_layout(title={'text': 'Revenue Trend','y':0.9,'x':0.5, 'xanchor': 'center','yanchor': 'top'},
                          legend=dict(yanchor="bottom",y=0.05,xanchor="right",x=0.95),autosize=True,margin=dict(t=70,b=0,l=0,r=0))
fig.show()

Revenue per Month

In [ ]:
revenue_per_month_df=df.groupby('Month', as_index=False )['TotalSales'].sum().sort_values(by="Month",ascending=True)
revenue_per_month_df.columns=['Month','Revenue']
revenue_per_month_df

In [ ]:
fig=px.bar(revenue_per_month_df,x='Month',y='Revenue',color='Revenue',text='Revenue',
title='Current Monthly Revenue Distribution')
fig.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.80),autosize=True,margin=dict(t=30,b=0,l=0,r=0))

Revenue Distribution on Day of Month

In [ ]:
revenue_per_day_df=df.groupby('Day', as_index=False )['TotalSales'].sum().sort_values(by="Day",ascending=True)
revenue_per_day_df.columns=['Day','Revenue']
revenue_per_day_df.head()

In [ ]:
fig=px.bar(revenue_per_day_df,x='Day',y='Revenue',color='Revenue',text='Revenue',title='Revenue Dist')
fig.update_layout(legend=dict(yanchor="top",y=0.95,xanchor="left",x=0.80),autosize=True,margin=dict(t=30,b=0,l=0,r=0))

## Prepare Data for Modeling

Transform Data to RFM (Frequency, Recency, Age and Monetary)

In [ ]:
last_order_date=df['Date'].max()
last_order_date

In [ ]:
lifetimes_txn_data = summary_data_from_transaction_data(df, 'CustomerID', 'Date', monetary_value_col='TotalSales', observation_period_end=last_order_date).reset_index()
lifetimes_txn_data=lifetimes_txn_data[lifetimes_txn_data['CustomerID']!='nan']

In [ ]:
lifetimes_txn_data.head()

Train BG/NBD Model

In [ ]:
bgf_model=BetaGeoFitter(penalizer_coef=0.0)
bgf_model.fit(lifetimes_txn_data['frequency'],lifetimes_txn_data['recency'],lifetimes_txn_data['T'])

In [ ]:
bgf_model.summary

Visualize Recency Frequency Matrix

In [ ]:
fig = plt.figure(figsize=(12,9))
plot_frequency_recency_matrix(bgf_model)

Visualize Probability of Customer Being Alive

In [ ]:
fig = plt.figure(figsize=(12,9))
plot_probability_alive_matrix(bgf_model)

Model Evaluation

In [ ]:
fig = plt.figure(figsize=(16,9))
plot_period_transactions(bgf_model)

Make Predictions with BG/NBD Model

Predict the customers who will make transactions in the next 10 days from the model

In [ ]:
t=12
lifetimes_txn_data['predicted_num_of_txns'] = round(bgf_model.conditional_expected_number_of_purchases_up_to_time(t, lifetimes_txn_data['frequency'], lifetimes_txn_data['recency'], lifetimes_txn_data['T']),2)
lifetimes_txn_data=lifetimes_txn_data.sort_values(by='predicted_num_of_txns', ascending=False)
lifetimes_txn_data.head(t)

Predict Customers Probability of Being Alive

In [ ]:
# days_since_birth = 200
# sp_trans = lifetimes_txn_data.loc[lifetimes_txn_data['CustomerID'] ==14911.0]
# plot_history_alive(bgf_model, days_since_birth, sp_trans, 'Date')

Use Gama-Gama Model to calculate CLV

We have to first check if the Pearson correlation between the frequency and monetary value is close to 0 before we can use the gama-gama model

In [ ]:
lifetimes_txn_data[['monetary_value', 'frequency']].corr()

In [ ]:
# Get customers with frequency >0
lifetimes_txn_data=lifetimes_txn_data[lifetimes_txn_data['frequency']>0]

In [ ]:
ggf_model = GammaGammaFitter(penalizer_coef = 0)
ggf_model.fit(lifetimes_txn_data['frequency'],lifetimes_txn_data['monetary_value'])

Predict Average Transaction Value for each Customer

In [ ]:
lifetimes_txn_data['predicted_value_of_txn'] = round(ggf_model.conditional_expected_average_profit(
        lifetimes_txn_data['frequency'],lifetimes_txn_data['monetary_value']), 2)
lifetimes_txn_data.head(10)

Calculate the Customer Lifetime Value (CLV)

In [ ]:
t=12 # number of months
rate=0.01 # monthly discount rate ~ 12.7% annually
lifetimes_txn_data['CLV'] = round(ggf_model.customer_lifetime_value(
    bgf_model, #the model to use to predict the number of future transactions
    lifetimes_txn_data['frequency'],
    lifetimes_txn_data['recency'],
    lifetimes_txn_data['T'],
    lifetimes_txn_data['monetary_value'],
    time=t,
    discount_rate=rate
), 2)

lifetimes_txn_data.columns=['Customer No.','Frequency','Recency','Age (T)','Monetary Value','Predicted No. of Txns','Predicted Value of Txns','Customer Lifetime Value (CLV)']

In [ ]:
lifetimes_txn_data.sort_values(by='Customer Lifetime Value (CLV)',ascending=False).head(10)

In [ ]:
revenue_per_customers_df=lifetimes_txn_data.groupby('Customer No.', as_index=False )['Customer Lifetime Value (CLV)'].sum().sort_values(by='Customer Lifetime Value (CLV)',ascending=False)
revenue_per_customers_df=revenue_per_customers_df[revenue_per_customers_df['Customer No.']!='nan']
fig=px.bar(revenue_per_customers_df.head(10),x='Customer No.',y='Customer Lifetime Value (CLV)',color='Customer No.',text='Customer Lifetime Value (CLV)',
title='Customer Lifetime Value for '+str(t)+' Months')
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.80),autosize=True,margin=dict(t=30,b=0,l=0,r=0))